In [70]:
# return a dataframe
from train_test_val import main
# return train, train_labels, test, test_labels, val, val_labels
from train_test_val import train_test_val_split

from sklearn.preprocessing import LabelEncoder, OrdinalEncoder
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.ensemble import RandomForestClassifier
from classification_utilities import display_cm, display_adj_cm
from sklearn.model_selection import train_test_split
import numpy as np
import pandas as pd
import os

path = r"C:\Users\Dunbar\Dropbox\Work\FORCE"
infile = "train.csv"
infile_path = os.path.join(path, infile)

In [63]:
df = main(infile_path, 51, 1.0)

Filling missing values from curves:
['RSHA', 'RMED', 'RDEP', 'RHOB', 'GR', 'NPHI', 'PEF', 'DTC']
----------------------------------------
Group by: ['WELL', 'FORMATION']
Total number of missing values: 1118313
Group by: ['WELL', 'GROUP']
Total number of missing values: 981495
Group by: ['WELL']
Total number of missing values: 415425
Group by: ['FORMATION']
Total number of missing values: 6057
Group by: ['GROUP']
Total number of missing values: 0


In [64]:
df["AI"] = df.DTC * df.RHOB

In [ ]:
def numericalise(df, string):
    _string = df[string].unique()
    _dict = dict(zip(_string, range(len(_string))))
    df = df.replace(_dict)
    return df

In [ ]:
df = numericalise(df, "FORMATION")

df = numericalise(df, "GROUP")

In [57]:
curves = ['DTC', 'FORMATION', 'GR', 'GROUP', 'NPHI',
          'PEF', 'RDEP', 'RHOB', 'RMED', 'RSHA', 'AI', 'FORCE_2020_LITHOFACIES_LITHOLOGY']
# curves = ['RSHA', 'RMED', 'RDEP', 'RHOB', 'GR', 'NPHI', 'PEF', 'DTC', 'AI']
# curves = ['RSHA', 'RMED', 'RDEP', 'GR', 'NPHI', 'PEF', 'AI']
# curves = ['RDEP', 'GR', 'NPHI', 'PEF', 'AI']

In [66]:
from sklearn.utils import shuffle
X = df[curves]
X = shuffle(X)

from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()

y = X.pop('FORCE_2020_LITHOFACIES_LITHOLOGY')

X = scaler.fit_transform(X)

In [67]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=42)

In [44]:
A = np.load('../penalty_matrix.npy')

def score(y_true, y_pred):
    S = 0.0
    y_true = y_true.astype(int)
    y_pred = y_pred.astype(int)
    for i in range(0, y_true.shape[0]):
        S -= A[y_true[i], y_pred[i]]
    return S/y_true.shape[0]


def make_model(model, X_train, y_train, X_test, y_test):
    
    model.fit(X_train, y_train)
    print("Accuracy on training set: {:.3f}".format(model.score(X_train, y_train)))
    print("Accuracy on test set: {:.3f}".format(model.score(X_test, y_test)))
    #print("Feature importances:\n{}".format(model.feature_importances_))
    predictions = model.predict(X_test)
    conf = confusion_matrix(y_test, predictions)
    print(display_cm(conf, facies_labels, hide_zeros=True))
    #print(confusion_matrix(y_test, predictions))
    print(score(y_test.values, predictions))

facies_labels = ['Sst', 'S-Sh', 'Sh', 'Mrl', 'Dol', 'Lst', 'Chk', 'Hal', 'Anh', 'Tuf', 'Coa', 'Bmt']

### Cons
> Sensitive to noisy data. It can overfit noisy data.
> The small variation(or variance) in data can result in the different decision tree. This can be reduced by bagging and boosting algorithms.
> Decision trees are biased with imbalance dataset, so it is recommended that balance out the dataset before creating the decision tree

In [41]:
model = DecisionTreeClassifier(random_state=0)
make_model(model, X_train, y_train, X_test, y_test)

Accuracy on training set: 0.999
Accuracy on test set: 0.893
Feature importances:
[0.05883239 0.06576221 0.18114419 0.0777926  0.09668004 0.07038338
 0.07142742 0.08718223 0.07709615 0.07846986 0.13522953]
     Pred   Sst  S-Sh    Sh   Mrl   Dol   Lst   Chk   Hal   Anh   Tuf   Coa   Bmt Total
     True
      Sst  8559   620   761    10     2    80                       6    36       10074
     S-Sh   594  7034   941    11     3   113                            15        8711
       Sh   731   969 38277   174    30   539     3     7     4    65    44     2 40845
      Mrl    19    16   161  1829     2   139     7                 2              2175
      Dol     1     2    44     2    45     2                 2                      98
      Lst    70   109   497   129     1  2782    43                24     1        3656
      Chk     1                 1          38   702                                 742
      Hal                 2           1               536     5                  

In [25]:
# 33% of data, default parameters
from sklearn.tree import DecisionTreeClassifier

model = DecisionTreeClassifier(random_state=0)
make_model(model, X_train, y_train, X_test, y_test)

Accuracy on training set: 0.999
Accuracy on test set: 0.896
Feature importances:
[0.06224893 0.07944436 0.176415   0.08315882 0.10038391 0.06604921
 0.06799717 0.08305034 0.06967084 0.07566848 0.13591293]
     Pred   Sst  S-Sh    Sh   Mrl   Dol   Lst   Chk   Hal   Anh   Tuf   Coa   Bmt Total
     True
      Sst  8681   596   797     7          61                       5    39       10186
     S-Sh   614  7159   895    18     4   103                       6    11        8810
       Sh   730   934 38301   165    37   529           3     5    53    51       40808
      Mrl    14     8   163  1892     1   108    10                                2196
      Dol     2     4    29     1    37     1                 1     3                78
      Lst    82    92   469   127        2825    45           2    16              3658
      Chk     3           2     2          37   617                                 661
      Hal                 4                           528     4                  

In [37]:
#33% of available data, of which 20% test
model = DecisionTreeClassifier(
    criterion='entropy', #"gini"
    splitter='best',  #'best'
    max_depth=30,#None,
    min_samples_split=2,
    min_samples_leaf=1,
    min_weight_fraction_leaf=0.0,
    max_features=None,
    random_state=None,
    max_leaf_nodes=None,
    min_impurity_decrease=0.0,
    min_impurity_split=None,
    class_weight=None,
    presort=False,
)
make_model(model, X_train, y_train, X_test, y_test)

Accuracy on training set: 0.998
Accuracy on test set: 0.899
Feature importances:
[0.07278738 0.07392914 0.20300383 0.12102948 0.09294807 0.06552012
 0.06387521 0.07262426 0.06419912 0.0746848  0.0953986 ]
     Pred   Sst  S-Sh    Sh   Mrl   Dol   Lst   Chk   Hal   Anh   Tuf   Coa   Bmt Total
     True
      Sst  8733   606   737     8     1    69                       4    28       10186
     S-Sh   591  7216   875     9     2   105                 2     1     9        8810
       Sh   754   866 38362   151    33   515     2     2     5    65    53       40808
      Mrl    10    11   174  1864     1   127     9                                2196
      Dol     1     4    28     1    40     1                 1     2                78
      Lst    76   107   463   123     1  2836    35           2    13     2        3658
      Chk     2           1     9          25   624                                 661
      Hal                 7                           527     2                  

In [61]:
#66% of available data, of which 10% test
model = DecisionTreeClassifier(
    criterion='entropy', #"gini"
    splitter='best',  #'best'
    max_depth=30,#None,
    min_samples_split=2,
    min_samples_leaf=1,
    min_weight_fraction_leaf=0.0,
    max_features=None,
    random_state=None,
    max_leaf_nodes=None,
    min_impurity_decrease=0.0,
    min_impurity_split=None,
    class_weight=None,
    presort=False,
)
make_model(model, X_train, y_train, X_test, y_test)

Accuracy on training set: 0.998
Accuracy on test set: 0.937
     Pred   Sst  S-Sh    Sh   Mrl   Dol   Lst   Chk   Hal   Anh   Tuf   Coa   Bmt Total
     True
      Sst  9189   370   446     3          46                       1    11       10066
     S-Sh   347  7856   551     7     3    55                 1           7        8827
       Sh   431   587 39257   103    25   278           3     1    33    29       40747
      Mrl     4     9    92  2012          86     3                                2206
      Dol     1     3    24          61                       3                      92
      Lst    58    54   363   102     2  3101    16                 6              3702
      Chk                 2     2          19   717                                 740
      Hal     1           5                           550                           556
      Anh                 4           2                 4    58                      68
      Tuf     1          30                12     

In [68]:
#100% of available data, of which 10% test
model = DecisionTreeClassifier(
    criterion='entropy', #"gini"
    splitter='best',  #'best'
    max_depth=30,#None,
    min_samples_split=2,
    min_samples_leaf=1,
    min_weight_fraction_leaf=0.0,
    max_features=None,
    random_state=None,
    max_leaf_nodes=None,
    min_impurity_decrease=0.0,
    min_impurity_split=None,
    class_weight=None,
    presort=False,
)
make_model(model, X_train, y_train, X_test, y_test)

Accuracy on training set: 0.996
Accuracy on test set: 0.952
     Pred   Sst  S-Sh    Sh   Mrl   Dol   Lst   Chk   Hal   Anh   Tuf   Coa   Bmt Total
     True
      Sst 14418   419   474     9     1    57                            18       15396
     S-Sh   417 12193   699     8     1    65                       1     7       13391
       Sh   484   680 60068    91    27   315     1     1     1    37    39       61744
      Mrl     7    12   109  3070     1    92     2                                3293
      Dol           3    29     5    89                             2               128
      Lst    49    74   401    98     2  4954    35           1     8              5622
      Chk                 2     6          23  1015                                1046
      Hal                 1                           830     1                     832
      Anh           1     3                             3   111                     118
      Tuf     3     2    32                10     

In [89]:
lithology_numbers = {30000: 0,
                 65030: 1,
                 65000: 2,
                 80000: 3,
                 74000: 4,
                 70000: 5,
                 70032: 6,
                 88000: 7,
                 86000: 8,
                 99000: 9,
                 90000: 10,
                 93000: 11}

In [72]:
open_test_features = pd.read_csv('../test.csv', sep=';')

# open_test_features.head()

print(open_test_features.columns)

open_test_features["AI"] = open_test_features.DTC * open_test_features.RHOB
open_test_features = numericalise(open_test_features, "FORMATION")
open_test_features = numericalise(open_test_features, "GROUP")
curves_test = ['DTC', 'FORMATION', 'GR', 'GROUP', 'NPHI',
               'PEF', 'RDEP', 'RHOB', 'RMED', 'RSHA', 'AI']
open_test_features = open_test_features[curves_test]
open_test_features

open_test_features = scaler.transform(open_test_features)
open_test_features

test_prediction = model.predict(open_test_features)

# test_prediction



category_to_lithology = {y:x for x,y in lithology_numbers.items()}

test_prediction_for_submission = np.vectorize(category_to_lithology.get)(test_prediction)

test_prediction_for_submission

np.savetxt('test_predictions_rf_entropy.csv', test_prediction_for_submission, header='lithology', comments='', fmt='%i')

Index(['WELL', 'DEPTH_MD', 'X_LOC', 'Y_LOC', 'Z_LOC', 'GROUP', 'FORMATION',
       'CALI', 'RSHA', 'RMED', 'RDEP', 'RHOB', 'GR', 'SGR', 'NPHI', 'PEF',
       'DTC', 'SP', 'BS', 'ROP', 'DTS', 'DCAL', 'DRHO', 'MUDWEIGHT', 'RMIC',
       'ROPA', 'RXO'],
      dtype='object')


In [42]:
model = DecisionTreeClassifier(
    criterion='entropy', #"gini"
    splitter='best',  #'best'
    max_depth=30,#None,
    min_samples_split=2,
    min_samples_leaf=1,
    min_weight_fraction_leaf=0.0,
    max_features=None,
    random_state=None,
    max_leaf_nodes=None,
    min_impurity_decrease=0.0,
    min_impurity_split=None,
    class_weight=None,
    presort=False,
)
make_model(model, X_train, y_train, X_test, y_test)

Accuracy on training set: 0.998
Accuracy on test set: 0.898
Feature importances:
[0.09079536 0.07873863 0.19720157 0.12533322 0.09160097 0.06734827
 0.06279793 0.07292925 0.06223172 0.06938503 0.08163806]
     Pred   Sst  S-Sh    Sh   Mrl   Dol   Lst   Chk   Hal   Anh   Tuf   Coa   Bmt Total
     True
      Sst  8613   604   733     9          68                       8    39       10074
     S-Sh   606  7104   872    18     4    89                       1    17        8711
       Sh   779   913 38370   147    25   482     3     5     2    68    49     2 40845
      Mrl    10    16   159  1852     1   131     5                 1              2175
      Dol           4    41     2    46     3                 2                      98
      Lst    67   101   483   123     1  2822    35                23     1        3656
      Chk                       1          34   707                                 742
      Hal                 5           1               535     3                  

In [48]:
from sklearn.ensemble import BaggingClassifier

bag_clf = BaggingClassifier(
    DecisionTreeClassifier(), n_estimators=1000,
    max_samples=20000, bootstrap=True, n_jobs=-1)

make_model(bag_clf, X_train, y_train, X_test, y_test)

Accuracy on training set: 0.873
Accuracy on test set: 0.851
     Pred   Sst  S-Sh    Sh   Mrl   Dol   Lst   Chk   Hal   Anh   Tuf   Coa   Bmt Total
     True
      Sst  8041   576  1427     8           5                      12     5       10074
     S-Sh  1129  5554  2001     5          14                       7     1        8711
       Sh  1177   639 38675   115     1   133     4     7     2    89     3       40845
      Mrl    21    26   379  1507         228    12                 2              2175
      Dol     3     7    72     2     6     1           4     2     1                98
      Lst   158   189   947   142        2120    60           2    38              3656
      Chk     1           4     1          78   658                                 742
      Hal                 6                           536     2                     544
      Anh                12                             8    43                      63
      Tuf     6          95                       

### Cross-validation

In [52]:
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold

In [53]:
kfold = KFold(n_splits=5)
scores = cross_val_score(DecisionTreeClassifier(criterion='entropy',
                                                splitter='best',
                                                max_depth=30),
                         X_train, y_train, cv=kfold)
print(f"Cross-validation scores: {scores}")

Cross-validation scores: [0.88530118 0.88429328 0.88599754 0.88797669 0.88507917]


In [59]:
#33% of data
kfold = KFold(n_splits=5)
scores = cross_val_score(DecisionTreeClassifier(criterion='entropy',
                                                splitter='best',
                                                max_depth=30),
                         X, y, cv=kfold)
print(f"Cross-validation scores: {scores}")

Cross-validation scores: [0.93375017 0.93162443 0.93272395 0.93265065 0.93166058]


In [62]:
#66% of data
kfold = KFold(n_splits=5)
scores = cross_val_score(DecisionTreeClassifier(criterion='entropy',
                                                splitter='best',
                                                max_depth=30),
                         X, y, cv=kfold)
print(f"Cross-validation scores: {scores}")

Cross-validation scores: [0.93338367 0.93144851 0.93264332 0.9323941  0.93151398]


In [83]:
#100% of data
kfold = KFold(n_splits=5)
scores = cross_val_score(DecisionTreeClassifier(criterion='entropy',
                                                splitter='best',
                                                max_depth=30),
                         X, y, cv=kfold)
print(f"Cross-validation scores: {scores}")

Cross-validation scores: [0.94852059 0.94855445 0.9485784  0.9481188  0.94752374]


In [84]:
from sklearn.ensemble import AdaBoostClassifier

In [87]:
ada_clf = AdaBoostClassifier(
    DecisionTreeClassifier(max_depth=30), n_estimators=1000,
    algorithm="SAMME.R", learning_rate=0.1)

ada_clf.fit(X_train, y_train)

AdaBoostClassifier(algorithm='SAMME.R',
                   base_estimator=DecisionTreeClassifier(class_weight=None,
                                                         criterion='gini',
                                                         max_depth=30,
                                                         max_features=None,
                                                         max_leaf_nodes=None,
                                                         min_impurity_decrease=0.0,
                                                         min_impurity_split=None,
                                                         min_samples_leaf=1,
                                                         min_samples_split=2,
                                                         min_weight_fraction_leaf=0.0,
                                                         presort=False,
                                                         random_state=None,
                            

In [88]:
print("Accuracy on training set: {:.3f}".format(ada_clf.score(X_train, y_train)))
print("Accuracy on test set: {:.3f}".format(ada_clf.score(X_test, y_test)))

predictions = ada_clf.predict(X_test)
conf = confusion_matrix(y_test, predictions)
print(display_cm(conf, facies_labels, hide_zeros=True))

print(score(y_test.values, predictions))

Accuracy on training set: 0.998
Accuracy on test set: 0.966
     Pred   Sst  S-Sh    Sh   Mrl   Dol   Lst   Chk   Hal   Anh   Tuf   Coa   Bmt Total
     True
      Sst 14739   277   332     3          31                            14       15396
     S-Sh   290 12500   537     8     1    50                       1     4       13391
       Sh   304   429 60701    71    14   177     1                21    26       61744
      Mrl     5     6    87  3132          62     1                                3293
      Dol     1     4    30     4    88                             1               128
      Lst    51    63   393    78     1  4999    26           1    10              5622
      Chk                 2     5          13  1026                                1046
      Hal                 2                           828     2                     832
      Anh           1     3                             4   110                     118
      Tuf     1          32                 6     

In [90]:
open_test_features = pd.read_csv('../test.csv', sep=';')

# open_test_features.head()

# print(open_test_features.columns)

open_test_features["AI"] = open_test_features.DTC * open_test_features.RHOB
open_test_features = numericalise(open_test_features, "FORMATION")
open_test_features = numericalise(open_test_features, "GROUP")
curves_test = ['DTC', 'FORMATION', 'GR', 'GROUP', 'NPHI',
               'PEF', 'RDEP', 'RHOB', 'RMED', 'RSHA', 'AI']
open_test_features = open_test_features[curves_test]
# print(open_test_features)

open_test_features = scaler.transform(open_test_features)
# print(open_test_features)

test_prediction = ada_clf.predict(open_test_features)

# print(test_prediction)

category_to_lithology = {y:x for x,y in lithology_numbers.items()}

test_prediction_for_submission = np.vectorize(category_to_lithology.get)(test_prediction)

test_prediction_for_submission

np.savetxt('test_predictions_ada-rf.csv', test_prediction_for_submission, header='lithology', comments='', fmt='%i')